The goal of this document is to analyse the vanishing behaviour of the polynomial $g_d(x) = x(x+1)\cdots(x+d-1)$ on certain finite rings. The context for asking this is that the polynomial $g_d(x) / d!$ is known to preserve integers, and I'd like to know when it also preserves *rings* of integers $\mathcal{O}_K$ of a number field $K$. For more on this, see
https://meer.gitlab.io/blog-5.html

In the function below, we decide whether or not a given polynomial $g_d(x) = x(x+1)\cdots(x+d-1)$ vanishes identically on the ring $\mathbb{Z}/p^N \mathbb{Z}[\varepsilon] / (\varepsilon^e)$. It uses two important optimisations.
 - The minimal $d$ for which $g_d(x)$ vanishes will be at least $pe$, and will be a multiple of $p$, so we do not bother to check any other value.
 - It suffices to check that $g_d(\varepsilon + n) = 0$ for $n = 0,\ldots,d$. This observation is particularly important, as it provides a performance increase that is exponential in $N$ and $e$. I'll expand on this point below.

In [ ]:
def g_d(d, x, n = 0):
    """Return the polynomial g_d(x) = (x+n)(x+n+1)...(x+n+d-1)."""
    g_d = 1
    for i in range(d):
        g_d *= (x + n + i)
    return g_d

In [ ]:
def is_zero(d, p, N, e):
    """Given a polynomial g_d(x) = x(x+1)...(x+d-1), test if g_d vanishes everywhere on the ring Z_{p^N}[epsilon] / (epsilon^e)."""
    #Construction of the ring of interest
    q = p**N
    R = PolynomialRing(IntegerModRing(q), 'x')
    x = R.gen()
    Q = R.quotient(x**e) #This is the ring we'll work with
    epsilon = Q.gen()
    
    #Construction of g_d
    g = g_d(d, x)
        
    #All we need to do is evaluate g_d on epsilon + n
    for n in range(d + 1):
        y = epsilon + n #We must evaluate at a instead of x to make clear to Sage that we want the outcome to land in Q
        if g(y) != 0:
            return False
    return True

def least_d(p, N, e):
    """Check if g_d(x) vanishes of Z_{p^N}[epsilon] / (epsilon^e) for d = 1,2,... until it does, and return that d."""
    d = p*e
    while True:
        if is_zero(d, p, N, e):
            return d
        d += p

Now let's crunch some numbers for the first couple of primes. This may take a little while.

In [ ]:
for p in [2, 3, 5, 7]:
    print("p =", p)
    l = [[least_d(p, N, e) for e in range(1, 16)] for N in range(1, 16)]
    print(*l, sep='\n')

Why does it suffice to evaluate $g_d(\varepsilon + n)$ for $n = 0,\ldots,d$? Because it turns out that the space of integer-preserving polynomials of degree $d$ is $(d + 1)$-dimensional, and so the remaining $g_d(\varepsilon + n)$ for $n > d$ are linearly dependent on the first $d + 1$ polynomials. We can illustrate this with Sage. Let's fix some $d$ and product $d + 2$ polynomials $g_d(x + n)$ for $n = 0,\ldots,d+1$. Then ask Sage whether we can find a linear combination of these polynomials that produces $0$.

In [ ]:
d = 5
V = QQ^(d + 1) #We have to work rationally for the Sage method to work, but the results are integral
#Construct g_d(x + n), take its coefficients. Use 'sparse = False' to include trailing zeros. Convert the list of coefficients into a vector. Assemble them into a list.
vectors = [vector(QQ, g_d(d, x, n).coefficients(sparse = False)) for n in range(d + 2)]
print(*V.linear_dependence(vectors))

The way you should read this is that 

$$ g_5(x) - 6g_5(x+1) + 15g_5(x+2) - 20g_5(x+3) + 15g_5(x+4) - 6g_5(x+5) + g_5(x+6) = 0. $$

Notice that these are coefficients are precisely $\pm {{n}\choose{6}}$. This is not a coincidence. One thing this implies is that there's a symmetry in the equations which makes me suspect that it suffices to evaluate $g_d(x + n)$ for $n = 0,\ldots,\lfloor d/2\rfloor$. 

In fact, for a moment I even suspected that we could optimise the program even further by merely evaluating $g_d(x)$ at just $x = \varepsilon + 1$. But that turns out not to be the case. As we'll see below, for $p = 3$, $N = 2$ and $e = 2$ we find examples for which the last nontrivial $g_d(x)$ vanishes on $\varepsilon + 1$.

In [ ]:
from itertools import product

p = 3
for (N, e) in product(range(1, 6), repeat = 2):
    d = least_d(p, N, e) - 1
    R = PolynomialRing(IntegerModRing(p**N), 'x')
    x = R.gen()
    Q = R.quotient(x**e)
    epsilon = Q.gen()
    g = g_d(d, x)
    print("(N, e) =", (N, e), "\t",
          "g(𝜀 + 1) =", g(epsilon + 1))

'xbar' is just what Sage calls the generator of the ring $Q = \mathbb{Z}_{p^N}[\varepsilon]$.

As said at the start, the main goal of this document was to understand more about the integer-preserving behaviour of the polynomials $g_d(x) / d!$ in a number field. Let's take some simple number fields and test the behaviour explicitly. We start off with some quadratic number fields. We fix $d = 3$ and we consider the fields $K = \mathbb{Q}(\sqrt{m})$ for $m$ between $-B$ and $B$. Which fields $K$ preserve $g_3(x) / 3!$?

In [ ]:
from itertools import product

d = 3
B = 100

def preserves_integers(g, K):
    """Given a polynomial g(x), check whether or not g preserves the ring of integers of K."""
    O_K = K.ideal(1)
    d = g.degree()
    B = O_K.integral_basis()
    for prod in product(range(d), repeat = len(B)):
        elt = 0
        for i in range(len(B)):
            elt += prod[i] * B[i]
        coords = O_K.coordinates(g(elt))
        if not all([x.is_integer() for x in coords]):
            return False
    return True

search_range = []
for i in range(-B, B + 1):
    if Integer(i).is_squarefree() and i != 1:
        search_range.append(i)

for m in search_range:
    x = polygen(QQ)
    K.<a> = NumberField(x^2 - m)
    g = g_d(d, x) / d.factorial()
    if preserves_integers(g, K):
        print(m)

Notice that all these numbers are equivalent to $1$ modulo $24$. This is not a coincidence: it turns out that $g_3(x) / 3!$ is well-defined on $\mathcal{O}_K$ if and only if $(2)$ and $(3)$ are completely split in $K$. Can we also find some cubics with this property? There are of course many more cubics than there are quadratics. But we can just generate some infinite family of them using Eisenstein polynomials.

In [ ]:
x = polygen(QQ)
for p in [5, 7, 11, 13, 17, 23]:
    for n in range(1, 101):
        f = x^3 + p*n*x + 6*p #Just some random Eisenstein polynomials
        K.<a> = NumberField(f)
        #K.decomposition_type(p) returns the decomposition behaviour of (p) in K in a particular format. We extract the splitting behaviour from it.
        if K.decomposition_type(2)[0][2] == 3 and K.decomposition_type(3)[0][2] == 3:
            print(f)

Let's take one and analyse it up close. Let's look at the precise splitting behaviour of $(2)$ and $(3)$.

In [ ]:
x = polygen(QQ)
K.<a> = NumberField(x^3 + 65*x + 30)
print(K.factor(2))
print(K.factor(3))

Now write $\mathcal{O}_K$ for the ring of integers of $K$. We can ask Sage to find an integral basis for it.

In [ ]:
O_K = K.ideal(1)
B = O_K.integral_basis()
B

We can express any element in terms of this integral basis. If the coordinates are themselves integers, then we know that the element is integral.

In [ ]:
g = g_d(3, x) / 3.factorial()

for prod in product(range(3), repeat = 3):
    elt = 0
    for i in range(3):
        elt += prod[i] * B[i]
    coords = O_K.coordinates(g(elt))
    print(prod, "-->", coords)

I ended up making the conjecture that $g_d(x) / d!$ is well-defined on $\mathcal{O}_K$ if and only if all primes $p \leq d$ are totally split in $K$. One evidence that the conjecture is nontrivial is that there are counterexamples when we take any multiple of this polynomial. Let's see why:

In [ ]:
g = g_d(7, x) / 7.factorial()

x = polygen(QQ)
for p in [11, 13, 17]:
    for n in range(1, 1001):
        f = x^3 + p*n*x + 600*p #Just some random Eisenstein polynomials
        K.<a> = NumberField(f)
        P = [preserves_integers(g, K), 
             preserves_integers(2*g, K), 
             preserves_integers(3*g, K), 
             preserves_integers(4*g, K),
             preserves_integers(5*g, K),
             preserves_integers(6*g, K)]
        if any(P):
            print(p, n, P)

The above script produces multiple examples of cubic fields for which $g_7(x) / 7!$ isn't well-defined, but $5 g_7(x) / 7!$ is. Although I was unable to find explicit examples, I would surely expect there to be number fields where only say $3 g_7(x)/7!$ is well-defined.